<a href="https://colab.research.google.com/github/21002268uhi/21002268_DataAnalytics/blob/main/oneHot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
library(tidyverse)
library(lubridate)
install.packages("mltools")
library(mltools)
library(data.table)


Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.4.0      ✔ purrr   0.3.5 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.3      ✔ forcats 0.5.2 

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: timechange


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


Attaching package: ‘mltools’


The following object is masked from ‘package:tidyr’:

    replace_na



Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, w

Loads the collated data from Github

In [ ]:
collated_data<- read.csv("https://raw.githubusercontent.com/21002268uhi/21002268_DataAnalytics/main/bquxjob_18ad732b_184bdcb4cd4.csv")

In [ ]:
latest_data<- read.csv('https://raw.githubusercontent.com/21002268uhi/21002268_DataAnalytics/main/bquxjob_146505aa_184d278d720.csv')

The function below is based on the one used in part 1 of the assignment but has had one hot coding added.

In [ ]:
#@title Function to clean and manipulate the data

clean_the_data<- function(df){
clean<- df%>%
  # select the variables
  select(collision_date, num_collisions: vehicle_5, temp, visib, wdsp, prcp,
         fog )%>%
           
  # rename date
  mutate(date = collision_date)%>%
  select(-collision_date)%>%

  # calculate numbers of vehicles involved in collisions
  mutate(year = year(date))%>%
  mutate(no_v = num_collisions-vehicle_1)%>%
  mutate(one_v = vehicle_1 - vehicle_2)%>%
  mutate(two_v = vehicle_2 - vehicle_3)%>%
  mutate(three_v = vehicle_3 - vehicle_4)%>%
  mutate(four_v = vehicle_4- vehicle_5)%>%
  mutate(five_v = vehicle_5)%>%
  select (-vehicle_1, -vehicle_2, -vehicle_3, -vehicle_4, -vehicle_5)%>%
 
 # Adds day of week
  mutate(day_of_week = as.factor(wday(date)))%>%
  mutate(isMon = ifelse(day_of_week == 1, 1,0),
          isTue = ifelse(day_of_week == 2, 1,0),
          isWed = ifelse(day_of_week == 3, 1,0),
          isThur = ifelse(day_of_week == 4, 1,0),
          isFri = ifelse(day_of_week == 5, 1,0),
          isSat = ifelse(day_of_week == 6, 1,0),
          isSun = ifelse(day_of_week == 7, 1,0))%>%

  # Adds year, month, day and removes date
  mutate(year = year(date))%>%
  mutate(month = month(date))%>%
  mutate(day = day(date))%>%
  select(-date)

  # Cleans the weather data
    # replace NA or 9999.9 in temp with average temp
    replace(clean$temp, clean$temp == 9999.9, median(clean$temp, na.rm = T))
    clean$temp[is.na(clean$temp)]<-median(clean$temp, na.rm = T)
    # visib replace missing with the max visibility of 10
    replace(clean$visib, clean$visib == 999.9, 10)
    clean$visib[is.na(clean$visib)]<-10
    # wdsp replacing missing and NA with mean
    replace(clean$wdsp, clean$wdsp == 999.9, median(clean$wdsp, na.rm = T))
    replace(clean$wdsp, clean$wdsp[is.na(clean$wdsp)], median(clean$wdsp, na.rm = T))
    # mxpsd replace missing with median 
    clean$mxpsd[clean$mxpsd == 999.9]<-median(clean$mxpsd, na.rm = T)
    replace(clean$mxpsd, clean$mxpsd[is.na(clean$mxpsd)], median(clean$mxpsd, na.rm = T))
    # gust replace missing with median 
    clean$gust[clean$gust == 999.9]<-median(clean$gust, na.rm = T)
    replace(clean$gust, clean$gust[is.na(clean$gust)], median(clean$gust, na.rm = T))
    # prcp missing = 99.99 should change this to zero
    clean$prcp[clean$prcp == 99.99]<-0
    replace(clean$prcp, clean$prcp[is.na(clean$prcp)], 0)
    # dewp missing and NA set to median
    clean$dewp[clean$dewp == 9999.9]<-median(clean$dewp, na.rm = T)
    replace(clean$dewp, clean$dewp[is.na(clean$dewp)],median(clean$dewp, na.rm = T) )
    # max missing and NA set to median
    clean$max[clean$max == 9999.9]<-median(clean$max, na.rm = T)
    replace(clean$max, clean$max[is.na(clean$max)],median(clean$max, na.rm = T) )
    # min missing and NA set to median
    clean$min[clean$min == 9999.9]<-median(clean$min, na.rm = T)
    replace(clean$min, clean$min[is.na(clean$min)],median(clean$min, na.rm = T) )

  # Remove the rows with missing data
    clean<- clean%>%
    filter(!is.na(prcp))%>%

  # Adds one hot coding
    mutate(year = as.factor(year),
          month = as.factor(month))

    clean <- one_hot(as.data.table(clean))

  return (clean)
}



In [ ]:
# runs the function to clean the data
ohc_clean_data <- clean_the_data(collated_data)
print(ohc_clean_data)

      num_collisions temp visib wdsp prcp fog year_2012 year_2013 year_2014
   1:            538 83.6   9.7  4.1 0.00   0         1         0         0
   2:            564 80.3  10.0  3.8 0.00   0         1         0         0
   3:            664 79.8  10.0  2.9 0.00   0         1         0         0
   4:            432 81.8   9.2  3.6 0.06   0         1         0         0
   5:            591 86.7   9.4  3.8 0.00   0         1         0         0
  ---                                                                      
3771:            293 65.6   6.6 10.9 2.14   1         0         0         0
3772:            284 55.3  10.0 17.5 0.00   0         0         0         0
3773:            293 55.1  10.0 10.3 0.00   0         0         0         0
3774:            279 53.8   9.6  4.9 0.00   1         0         0         0
3775:            262 49.3   9.9  4.4 0.00   0         0         0         0
      year_2015 year_2016 year_2017 year_2018 year_2019 year_2020 year_2021
   1:       

In [ ]:
# runs function to clean latest data
ohc_latest<- clean_the_data(latest_data)

# adds the missing variables
ohc_latest<-ohc_latest%>%
  mutate( year_2021 = 0, year_2020 = 0, year_2019 = 0, year_2018 = 0, year_2017 = 0,
          year_2016 = 0, year_2015 = 0, year_2014 = 0, year_2013 = 0, year_2012 = 0)%>%
  mutate( month_1 = 0, month_2 = 0, month_3 = 0, month_4 = 0, month_5 = 0, month_6 = 0,
          month_7 = 0, month_8 = 0, month_9 = 0, month_10 = 0, month_11 = 1, month_12 = 0)

print(ohc_latest)

   num_collisions temp visib wdsp prcp fog year_2022 no_v one_v two_v three_v
1:            153 43.5  10.0  6.9 0.00   0         1    0    42    92      12
2:            188 54.9   9.6 11.6 0.00   0         1    2    42   126      14
3:            268 48.7  10.0 14.9 0.00   0         1    1    53   189      18
4:            278 36.9  10.0 16.1 0.00   0         1    0    65   195      15
5:            197 47.2  10.0 16.2 0.11   0         1    3    54   112      19
6:            257 50.2  10.0  9.4 0.00   0         1    1    52   176      18
7:            195 52.6   9.3 10.3 0.00   0         1    3    39   129      16
   four_v five_v day_of_week_1 day_of_week_2 day_of_week_3 day_of_week_4
1:      5      2             0             0             0             0
2:      1      3             1             0             0             0
3:      4      3             0             0             1             0
4:      1      2             0             1             0             0
5:      9  

In [ ]:
#@title Creates function to remove outliers 

remove_outliers<- function(df){
  #make new dataset that will have outliers removed
  clean_no_outliers<- df

  # remove outliers from wdsp
  outliers <- boxplot(clean_no_outliers$wdsp, plot=FALSE)$out
  clean_no_outliers <- clean_no_outliers[-which(clean_no_outliers$wdsp %in% outliers),]

  # remove outliers from temp
  outliers <- boxplot(clean_no_outliers$temp, plot=FALSE)$out
  clean_no_outliers <- clean_no_outliers[-which(clean_no_outliers$temp %in% outliers),]

  # remove outliers from visib
  outliers <- boxplot(clean_no_outliers$visib, plot=FALSE)$out
  clean_no_outliers <- clean_no_outliers[-which(clean_no_outliers$visib %in% outliers),]

  # remove outliers from prcp
  outliers <- boxplot(clean_no_outliers$prcp, plot=FALSE)$out
  clean_no_outliers <- clean_no_outliers[-which(clean_no_outliers$prcp %in% outliers),]

  return(clean_no_outliers)
}


In [ ]:
# runs the function to remove outliers
ohc_no_outliers<- remove_outliers(ohc_clean_data)

In [ ]:
# Save the data files to use later
write.csv(ohc_clean_data, file = "ohc_clean.csv")
write.csv(ohc_no_outliers, file = "ohc_no_outliers.csv")
write.csv(ohc_latest, file = "ohc_latest.csv")